## Transformer

In [62]:
import random

# Parameters
num_lists = 100  # Number of inner lists
list_length = 6  # Number of integers in each inner list
min_val, max_val = 0, 9  # Range for integers

# Generate the list of lists
dataset = [[7]+[random.randint(min_val, max_val) for _ in range(list_length)] for _ in range(num_lists)]

print(dataset)

[[7, 8, 5, 7, 3, 2, 9], [7, 6, 4, 1, 9, 6, 0], [7, 6, 3, 2, 5, 9, 6], [7, 8, 5, 9, 5, 9, 3], [7, 7, 0, 2, 5, 7, 9], [7, 3, 5, 5, 6, 1, 0], [7, 2, 7, 2, 7, 0, 6], [7, 3, 6, 1, 2, 5, 8], [7, 2, 9, 1, 4, 4, 2], [7, 2, 5, 5, 2, 8, 1], [7, 1, 7, 9, 2, 3, 8], [7, 2, 0, 5, 1, 9, 0], [7, 3, 0, 3, 4, 7, 2], [7, 8, 3, 3, 4, 0, 4], [7, 8, 9, 2, 7, 9, 9], [7, 3, 2, 5, 5, 5, 4], [7, 1, 8, 2, 1, 9, 0], [7, 5, 8, 6, 0, 8, 4], [7, 4, 7, 4, 9, 7, 2], [7, 6, 7, 1, 5, 2, 9], [7, 5, 8, 8, 1, 8, 0], [7, 8, 1, 4, 6, 6, 0], [7, 2, 0, 5, 3, 3, 9], [7, 5, 3, 4, 3, 3, 8], [7, 3, 0, 3, 2, 8, 7], [7, 3, 3, 7, 0, 7, 9], [7, 6, 4, 4, 2, 1, 1], [7, 6, 4, 0, 0, 0, 6], [7, 9, 9, 8, 4, 6, 8], [7, 3, 2, 3, 4, 7, 9], [7, 6, 6, 3, 4, 8, 4], [7, 4, 0, 2, 1, 5, 8], [7, 5, 0, 0, 9, 4, 9], [7, 8, 3, 2, 4, 9, 8], [7, 8, 3, 6, 1, 2, 6], [7, 7, 6, 6, 4, 2, 2], [7, 8, 5, 6, 1, 3, 2], [7, 4, 8, 6, 4, 9, 8], [7, 5, 6, 8, 7, 7, 3], [7, 7, 8, 6, 5, 9, 9], [7, 1, 0, 9, 8, 7, 4], [7, 8, 3, 5, 6, 2, 0], [7, 0, 5, 4, 0, 2, 7], [7, 6, 7, 

In [63]:
import torch
import torch.nn as nn


class SelfAttention(nn.Module):
    def __init__(self, embed_size, heads):
        super(SelfAttention, self).__init__()
        self.embed_size = embed_size
        self.heads = heads
        self.head_dim = embed_size // heads

        assert (
            self.head_dim * heads == embed_size
        ), "Embedding size needs to be divisible by heads"

        self.values = nn.Linear(embed_size, embed_size)
        self.keys = nn.Linear(embed_size, embed_size)
        self.queries = nn.Linear(embed_size, embed_size)
        self.fc_out = nn.Linear(embed_size, embed_size)

    def forward(self, values, keys, query, mask):
        # Get number of training examples
        N = query.shape[0]

        value_len, key_len, query_len = values.shape[1], keys.shape[1], query.shape[1]

        values = self.values(values)  # (N, value_len, embed_size)
        keys = self.keys(keys)  # (N, key_len, embed_size)
        queries = self.queries(query)  # (N, query_len, embed_size)

        # Split the embedding into self.heads different pieces
        values = values.reshape(N, value_len, self.heads, self.head_dim)
        keys = keys.reshape(N, key_len, self.heads, self.head_dim)
        queries = queries.reshape(N, query_len, self.heads, self.head_dim)

        # Einsum does matrix mult. for query*keys for each training example
        # with every other training example, don't be confused by einsum
        # it's just how I like doing matrix multiplication & bmm

        energy = torch.einsum("nqhd,nkhd->nhqk", [queries, keys])
        # queries shape: (N, query_len, heads, heads_dim),
        # keys shape: (N, key_len, heads, heads_dim)
        # energy: (N, heads, query_len, key_len)

        # Mask padded indices so their weights become 0
        if mask is not None:
            energy = energy.masked_fill(mask == 0, float("-1e20"))

        # Normalize energy values similarly to seq2seq + attention
        # so that they sum to 1. Also divide by scaling factor for
        # better stability
        attention = torch.softmax(energy / (self.embed_size ** (1 / 2)), dim=3)
        # attention shape: (N, heads, query_len, key_len)

        out = torch.einsum("nhql,nlhd->nqhd", [attention, values]).reshape(
            N, query_len, self.heads * self.head_dim
        )
        # attention shape: (N, heads, query_len, key_len)
        # values shape: (N, value_len, heads, heads_dim)
        # out after matrix multiply: (N, query_len, heads, head_dim), then
        # we reshape and flatten the last two dimensions.

        out = self.fc_out(out)
        # Linear layer doesn't modify the shape, final shape will be
        # (N, query_len, embed_size)

        return out


class TransformerBlock(nn.Module):
    def __init__(self, embed_size, heads, dropout, forward_expansion):
        super(TransformerBlock, self).__init__()
        self.attention = SelfAttention(embed_size, heads)
        self.norm1 = nn.LayerNorm(embed_size)
        self.norm2 = nn.LayerNorm(embed_size)

        self.feed_forward = nn.Sequential(
            nn.Linear(embed_size, forward_expansion * embed_size),
            nn.ReLU(),
            nn.Linear(forward_expansion * embed_size, embed_size),
        )

        self.dropout = nn.Dropout(dropout)

    def forward(self, value, key, query, mask):
        attention = self.attention(value, key, query, mask)

        # Add skip connection, run through normalization and finally dropout
        x = self.dropout(self.norm1(attention + query))
        forward = self.feed_forward(x)
        out = self.dropout(self.norm2(forward + x))
        return out


class Encoder(nn.Module):
    def __init__(
        self,
        src_vocab_size,
        embed_size,
        num_layers,
        heads,
        device,
        forward_expansion,
        dropout,
        max_length,
    ):

        super(Encoder, self).__init__()
        self.embed_size = embed_size
        self.device = device
        self.word_embedding = nn.Embedding(src_vocab_size, embed_size)
        self.position_embedding = nn.Embedding(max_length, embed_size)

        self.layers = nn.ModuleList(
            [
                TransformerBlock(
                    embed_size,
                    heads,
                    dropout=dropout,
                    forward_expansion=forward_expansion,
                )
                for _ in range(num_layers)
            ]
        )

        self.dropout = nn.Dropout(dropout)

    def forward(self, x, mask):
        N, seq_length = x.shape
        positions = torch.arange(0, seq_length).expand(N, seq_length).to(self.device)
        out = self.dropout(
            (self.word_embedding(x) + self.position_embedding(positions))
        )

        # In the Encoder the query, key, value are all the same, it's in the
        # decoder this will change. This might look a bit odd in this case.
        for layer in self.layers:
            out = layer(out, out, out, mask)

        return out


class DecoderBlock(nn.Module):
    def __init__(self, embed_size, heads, forward_expansion, dropout, device):
        super(DecoderBlock, self).__init__()
        self.norm = nn.LayerNorm(embed_size)
        self.attention = SelfAttention(embed_size, heads=heads)
        self.transformer_block = TransformerBlock(
            embed_size, heads, dropout, forward_expansion
        )
        self.dropout = nn.Dropout(dropout)

    def forward(self, x, value, key, src_mask, trg_mask):
        attention = self.attention(x, x, x, trg_mask)
        query = self.dropout(self.norm(attention + x))
        out = self.transformer_block(value, key, query, src_mask)
        return out


class Decoder(nn.Module):
    def __init__(
        self,
        trg_vocab_size,
        embed_size,
        num_layers,
        heads,
        forward_expansion,
        dropout,
        device,
        max_length,
    ):
        super(Decoder, self).__init__()
        self.device = device
        self.word_embedding = nn.Embedding(trg_vocab_size, embed_size)
        self.position_embedding = nn.Embedding(max_length, embed_size)

        self.layers = nn.ModuleList(
            [
                DecoderBlock(embed_size, heads, forward_expansion, dropout, device)
                for _ in range(num_layers)
            ]
        )
        self.fc_out = nn.Linear(embed_size, trg_vocab_size)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x, enc_out, src_mask, trg_mask):
        N, seq_length = x.shape
        positions = torch.arange(0, seq_length).expand(N, seq_length).to(self.device)
        x = self.dropout((self.word_embedding(x) + self.position_embedding(positions)))

        for layer in self.layers:
            x = layer(x, enc_out, enc_out, src_mask, trg_mask)

        out = self.fc_out(x)

        return out


class Transformer(nn.Module):
    def __init__(
        self,
        src_vocab_size,
        trg_vocab_size,
        src_pad_idx,
        trg_pad_idx,
        embed_size=512,
        num_layers=6,
        forward_expansion=4,
        heads=8,
        dropout=0,
        device="cpu",
        max_length=100,
    ):

        super(Transformer, self).__init__()

        self.encoder = Encoder(
            src_vocab_size,
            embed_size,
            num_layers,
            heads,
            device,
            forward_expansion,
            dropout,
            max_length,
        )

        self.decoder = Decoder(
            trg_vocab_size,
            embed_size,
            num_layers,
            heads,
            forward_expansion,
            dropout,
            device,
            max_length,
        )

        self.src_pad_idx = src_pad_idx
        self.trg_pad_idx = trg_pad_idx
        self.device = device

    def make_src_mask(self, src):
        src_mask = (src != self.src_pad_idx).unsqueeze(1).unsqueeze(2)
        # (N, 1, 1, src_len)
        return src_mask.to(self.device)

    def make_trg_mask(self, trg):
        N, trg_len = trg.shape
        trg_mask = torch.tril(torch.ones((trg_len, trg_len))).expand(
            N, 1, trg_len, trg_len
        )

        return trg_mask.to(self.device)

    def forward(self, src, trg):
        src_mask = self.make_src_mask(src)
        trg_mask = self.make_trg_mask(trg)
        enc_src = self.encoder(src, src_mask)
        out = self.decoder(trg, enc_src, src_mask, trg_mask)
        return out


if __name__ == "__main__":
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(device)

    x = torch.tensor(dataset).to(
        device
    )
    trg = torch.tensor(dataset).to(device)

    src_pad_idx = 0
    trg_pad_idx = 0
    src_vocab_size = 10
    trg_vocab_size = 10
    model = Transformer(src_vocab_size, trg_vocab_size, src_pad_idx, trg_pad_idx, device=device).to(
        device
    )
    out = model(x, trg[:, :-1])
    print(out.shape)

cpu
torch.Size([100, 6, 10])


In [64]:
# Define the loss function (CrossEntropyLoss for classification tasks)
criterion = nn.CrossEntropyLoss(ignore_index=trg_pad_idx)  # Ignore padding tokens during loss calculation

# Optimizer (Adam is commonly used)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)


In [65]:
num_epochs = 100  # Set number of epochs for training

for epoch in range(num_epochs):
    model.train()  # Set model to training mode
    
    optimizer.zero_grad()  # Clear the gradients from the previous step

    # Forward pass
    output = model(x, trg[:, :-1])  # Pass input (x) and target (trg) through the model
    
    # Calculate the loss (only consider the target sequence's tokens)
    output = output.reshape(-1, trg_vocab_size)  # Flatten the output for loss calculation
    trg_out = trg[:, 1:].reshape(-1)  # Flatten target to match output shape
    loss = criterion(output, trg_out)  # Compute loss
    
    # Backward pass
    loss.backward()
    
    # Optimizer step (update model parameters)
    optimizer.step()
    
    # Print the loss at regular intervals
    if epoch % 10 == 0:
        print(f"Epoch [{epoch}/{num_epochs}], Loss: {loss.item()}")


Epoch [0/100], Loss: 2.415189504623413
Epoch [10/100], Loss: 1.7285680770874023
Epoch [20/100], Loss: 1.2294769287109375
Epoch [30/100], Loss: 0.6839950680732727
Epoch [40/100], Loss: 0.12359867244958878
Epoch [50/100], Loss: 0.011871354654431343
Epoch [60/100], Loss: 0.0028504030779004097
Epoch [70/100], Loss: 0.0014714812859892845
Epoch [80/100], Loss: 0.0010462398640811443
Epoch [90/100], Loss: 0.0008685660432092845


In [ ]:
model.eval()  # Set model to evaluation mode

# Example input for prediction (use your trained model)
# 7 signify start
test_input = torch.tensor([[7,1,3,2,4,5,3]], device=device)  # Example input batch
predictions = model(test_input, test_input[:, :-1])

# Get the predicted token indices
predicted_tokens = torch.argmax(predictions, dim=-1)

print(predicted_tokens)


tensor([[1, 3, 2, 4, 5, 3]])
